In [2]:
%reset -f

# Deep Learning용 기본 import 및 seed 설정

# 필요한 module import
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime

# Random Seed를 위해서 포함
import os
import random

# TensorFlow
import tensorflow as tf  
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Flatten, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import TensorBoard

os.environ["CUBLAS_WORKSPACE_CONFIG"] = ":4096:8"

# PyTorch
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.tensorboard import SummaryWriter

# 시드 설정하는 함수
def seed_everything(seed=42):

    # 1. 환경변수
    os.environ['PYTHONHASHSEED'] = str(seed)

    # 2. Python, Numpy
    random.seed(seed)
    np.random.seed(seed)

    # 3. TensorFlow
    os.environ['TF_DETERMINISTIC_OPS'] = '1'
    tf.random.set_seed(seed)

    # 4. PyTorch
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    torch.use_deterministic_algorithms(True)  # PyTorch 1.8+ 권장

    print(f"Seeds set to {seed}")

seed_everything(42)

2025-11-11 12:02:55.283885: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-11-11 12:02:55.460753: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:479] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-11-11 12:02:55.543407: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:10575] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-11-11 12:02:55.544080: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1442] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-11-11 12:02:55.666358: I tensorflow/core/platform/cpu_feature_gua

Seeds set to 42


In [3]:
# 데이터 준비
# AND, OR, XOR 게이트를 학습할 수 있는지 확인할꺼예요!
x_data = np.array([[0,0],
                   [0,1],
                   [1,0],
                   [1,1]], dtype=np.float32)
# AND Gate
y_data = np.array([0,0,0,1], dtype=np.float32).reshape(-1,1)



In [5]:
# TensorFlow 구현
# Model 생성
keras_model = Sequential()

# Model에 Layer를 추가
keras_model.add(Input(shape=(2,)))
keras_model.add(Dense(units=1,
                      activation='sigmoid'))
# Model Compile
keras_model.compile(optimizer=Adam(learning_rate=1e-1),
                    loss='binary_crossentropy',
                    metrics=['accuracy'])

# TensorBoard callback
log_dir = './logs/' + datetime.datetime.now().strftime('%Y%m%d-%H%M%S')
tb_cb = TensorBoard(log_dir=log_dir,
                    histogram_freq=1)

# 학습을 진행
keras_model.fit(x_data,
                y_data,
                epochs=500,
                callbacks=[tb_cb],
                verbose=0)


In [6]:
# 학습이 제대로 되었는지 확인(평가)

# 평가데이터가 따로 없어요. 그래서 학습데이터로 평가해서
# 정상적으로 학습이 되었는지 확인!
keras_y_pred = keras_model.predict(x_data)
keras_y_pred_class = (keras_y_pred >= 0.5).astype(int)

from sklearn.metrics import accuracy_score
accuracy_score(y_data,
               keras_y_pred_class)
# AND, OR => 1.0
# XOR => 0.25



1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step


1.0

In [8]:
# PyTorch 구현

# for TensorBoard
log_dir = './logs/' + datetime.datetime.now().strftime('%Y%m%d-%H%M%S')
wirter = SummaryWriter(log_dir=log_dir)

# for GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Trainig Tensor(PyTorch Tensor로 변환)
x_tensor = torch.FloatTensor(x_data)
y_tensor = torch.FloatTensor(y_data)

# Model class 정의
class PerceptronClass(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(2,1)  # TensorFlow의 Dense()
    def forward(self,x):
        # 순전파 구조(Feedforward) vs 순전파 연산(Forward Propagation)
        # 순전파 하는 함수
        # 우리 모델에 데이터를 넣어서 예측값을 얻어내는 연산과정
        out = self.linear(x)
        out = torch.sigmoid(out)
        return out

# Model 생성
torch_model = PerceptronClass()

# Loss와 Optimizer 생성
criterion = nn.BCELoss()
optimizer = optim.Adam(torch_model.parameters(),
                       lr=1e-3)

# 훈련을 진행
epochs=20000

# 학습을 진행(backpropagation을 이용한 가중치 조절)
torch_model.train()

for epoch in range(epochs):
    # 순전파
    y_pred = torch_model(x_tensor)
    # Loss 계산
    loss = criterion(y_pred, y_tensor)
    # loss : 정답과 모델 예측값의 차이!

    # 역전파를 통한 가중치 update
    optimizer.zero_grad() # 기존 기울기 초기화
    loss.backward()   # 기울기 역전파
    optimizer.step()  # 가중치 update

    # TensorBoard용 로그파일에 내용 기록
    wirter.add_scalar('Loss/train',
                      loss.item(), epoch)

    # 학습이 진행되는지 확인하기 위한 출력구문
    if epoch % 2000 == 0:
        print(f'Epoch : {epoch}/{epochs}, Loss : {loss.item()}')



Epoch : 0/20000, Loss : 0.7610647678375244
Epoch : 2000/20000, Loss : 0.406097412109375
Epoch : 4000/20000, Loss : 0.24944382905960083
Epoch : 6000/20000, Loss : 0.157632514834404
Epoch : 8000/20000, Loss : 0.09901855885982513
Epoch : 10000/20000, Loss : 0.061659228056669235
Epoch : 12000/20000, Loss : 0.03820893540978432
Epoch : 14000/20000, Loss : 0.02362879179418087
Epoch : 16000/20000, Loss : 0.014600981026887894
Epoch : 18000/20000, Loss : 0.009020705707371235


In [9]:
# 우리 모델을 평가해야 해요!
torch_model.eval()

from sklearn.metrics import accuracy_score

with torch.no_grad():
    torch_y_pred = torch_model(x_tensor)
    torch_y_pred_class = (torch_y_pred.numpy() >= 0.5).astype(int)

    print(accuracy_score(y_data, torch_y_pred_class)) # 0.5

1.0


In [ ]:
# 결론적으로
# AND, OR 이런 Gate 진리표는 1개의 Perceptron(Logistic)으로 학습이 가능
# 그러나 XOR같은 이런 Gate의 진리표는 1개의 perceptron으로
# 학습이 안되요!